## Dependencias

In [1]:
import sys

#!{sys.executable} -m pip install torchio
#!{sys.executable} -m pip install scikit-image
#!{sys.executable} -m pip install kornia
#!{sys.executable} -m pip install h5py

In [2]:
import torch
import numpy as np

from skimage import measure

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

MODEL_NAME = 'unet6t'
FILENAME = MODEL_NAME + '.pth'
DIM_SIZE_REDUCTION = (1,1,1)
MODE = 'target'

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from utils import compare_output, metrics, draw_images, plot_epochs

CUDA is available!  Training on GPU ...


## Dataset

In [3]:
from dataset import CellsDataset

# DataLoader

In [4]:
from torchvision import datasets, transforms
import torchio.transforms as transformsio
from torch.utils.data.sampler import SubsetRandomSampler
import kornia.augmentation as K
import torch.nn as nn

from dataset import PATHS

train_path = PATHS['LQ_TRAIN']
valid_path = PATHS['LQ_VALID']
test_path = PATHS['LQ_TEST']
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 1

# convert data to a normalized torch.FloatTensor
transform = transforms.Compose([transformsio.ZNormalization()])
transform_augmentation = nn.Sequential(
    K.RandomDepthicalFlip3D(same_on_batch=True), 
    K.RandomHorizontalFlip3D(same_on_batch=True), 
    K.RandomVerticalFlip3D(same_on_batch=True),
#    K.RandomRotation3D((0.5, 3, 3), same_on_batch=True)
)

# choose the training and test datasets
train_data = CellsDataset(train_path, target_mode=MODE, transform=transform, transform_augmentation=transform_augmentation, dim_size_reduction=DIM_SIZE_REDUCTION)
valid_data = CellsDataset(valid_path, target_mode=MODE, transform=transform, dim_size_reduction=DIM_SIZE_REDUCTION)
test_data = CellsDataset(test_path, target_mode=MODE, transform=transform, dim_size_reduction=DIM_SIZE_REDUCTION)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, num_workers=num_workers)


If you use TorchIO for your research, please cite the following paper:
Pérez-García et al., TorchIO: a Python library for efficient loading,
preprocessing, augmentation and patch-based sampling of medical images
in deep learning. Credits instructions: https://torchio.readthedocs.io/#credits



## Modelos

Arquitectura U-Net reducida. Pensada para hacer entrenamientos rápidos para hacer pruebas en las que la calidad del resultado no importe.

In [5]:
from models import MiniUNet3D

Arquitectura U-Net completa.

In [6]:
from models import UNet3D

## Entrenamiento

Instanciación del modelo. Si se usa CUDA esperar unos segundos a que el modelo se cargue en GPU.

In [7]:
model = UNet3D(1,2)
if train_on_gpu:
    model.cuda()

Optimizador y definición de funciones de pérdida. También se usa 

In [9]:
import torch.optim as optim
#from apex import amp

optimizer = optim.Adam(model.parameters(), weight_decay=0.00001)

# https://github.com/mcarilli/mixed_precision_references/blob/master/Pytorch_Devcon_2019/devcon_2019_mcarilli_final.pdf

#model, optimizer = amp.initialize(model, optimizer, opt_level="O1")

def target_to_one_hot(target):
    temp = torch.reshape(target, (-1,)).long()
    target = torch.zeros([torch.numel(temp), 2])
    target[torch.arange(torch.numel(temp)),temp] = 1
    return target

from losses import simple_dice_loss3D, WeightedCrossEntropyLoss

Algoritmo de entrenamiento.

In [ ]:
import time
from torch.autograd import Variable
import os
import numpy as np
import torch.nn as nn
# número de epochs para entrenar el modelo
n_epochs = 400

# wce, dice
LOSS_FUNCTION = 'dice'
SAVE_MODEL = True

if LOSS_FUNCTION == 'wce':
    criterion = WeightedCrossEntropyLoss()

exists_best_model = False
if os.path.isfile(FILENAME):
    checkpoint = torch.load(FILENAME)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    train_losses = checkpoint['train_losses']
    valid_losses = checkpoint['valid_losses']
    current_epoch = checkpoint['epochs']
    best_model_state_dict = checkpoint['best_model_state_dict']
    best_optimizer_state_dict = checkpoint['optimizer_state_dict']
    valid_loss_min = checkpoint['valid_loss_min']
    #amp.load_state_dict(checkpoint['amp_state_dict'])
    exists_best_model = True
else:
    train_losses = []
    valid_losses = []
    current_epoch = 0
    valid_loss_min = np.Inf

start_training = time.time()
for epoch in range(current_epoch+1, current_epoch + n_epochs + 1):
    start_epoch = time.time()
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    ###################
    # train the model #
    ###################
    model.train()
    for data, target, correct_cell_count, resized_cell_count in train_loader:
        target = target.squeeze(0)
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data = Variable(data).cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        datasize = data.size(0)
        del data
        if LOSS_FUNCTION == 'dice':
            target = target_to_one_hot(target).float()
            if train_on_gpu:
                target = Variable(target).cuda()
            # calculate the batch loss
            criterion1 = nn.Softmax(dim=1)
            output = output.permute(0,2,3,4,1).contiguous().view(-1,2).float()
            loss = simple_dice_loss3D(criterion1(output), target)
        elif LOSS_FUNCTION == 'wce':
            if train_on_gpu:
                target = Variable(target).cuda().long()
            loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        #with amp.scale_loss(loss, optimizer) as scaled_loss:
        #    scaled_loss.backward()
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item() * datasize
        del target
        del output
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target, correct_cell_count, resized_cell_count in valid_loader:
        target = target.squeeze(0)
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data = Variable(data).cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        datasize = data.size(0)
        del data
        if LOSS_FUNCTION == 'dice':
            target = target_to_one_hot(target).float()
            if train_on_gpu:
                target = Variable(target).cuda()
            # calculate the batch loss
            criterion1 = nn.Softmax(dim=1)
            output = output.permute(0,2,3,4,1).contiguous().view(-1,2).float()
            loss = simple_dice_loss3D(criterion1(output), target)
        elif LOSS_FUNCTION == 'wce':
            if train_on_gpu:
                target = Variable(target).cuda().long()
            loss = criterion(output, target)
        del target
        del output
        # update average validation loss
        valid_loss += loss.item() * datasize
    # calculate average losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    # print training/validation statistics
    print('Epoch: {} Tiempo:{:.0f}s \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, time.time()-start_epoch, train_loss, valid_loss))
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        best_model_state_dict = model.state_dict()
        best_optimizer_state_dict = optimizer.state_dict()
        print('Validation loss decreased. Train loss: {:.6f} Validation Loss: ({:.6f} --> {:.6f}).  Saving model ...'.format(
        train_loss,
        valid_loss_min,
        valid_loss,
        ))
        valid_loss_min = valid_loss
        exists_best_model = True
    if exists_best_model:
        torch.save({
            'epochs': epoch,
            'best_model_state_dict': best_model_state_dict,
            'best_optimizer_state_dict' : best_optimizer_state_dict,
            'model_state_dict' : model.state_dict(),
            'optimizer_state_dict' : optimizer.state_dict(),
            'train_losses': train_losses,
            'valid_losses': valid_losses,
            'valid_loss_min': valid_loss_min,
            #'amp_state_dict': amp.state_dict()
        }, FILENAME)

print("-----")

plot_epochs(train_losses, valid_losses, MODEL_NAME)
metrics(model, test_data, save=True, model_name=MODEL_NAME)

print("Entrenamiento terminado en {:.2f}m".format((time.time() - start_training)/60))

Epoch: 1 Tiempo:38s 	Training Loss: 0.753476 	Validation Loss: 0.718838
Validation loss decreased. Train loss: 0.753476 Validation Loss: (inf --> 0.718838).  Saving model ...
Epoch: 2 Tiempo:38s 	Training Loss: 0.700408 	Validation Loss: 0.719370
Epoch: 3 Tiempo:37s 	Training Loss: 0.685912 	Validation Loss: 0.701134
Validation loss decreased. Train loss: 0.685912 Validation Loss: (0.718838 --> 0.701134).  Saving model ...
Epoch: 4 Tiempo:38s 	Training Loss: 0.682838 	Validation Loss: 0.705787
Epoch: 5 Tiempo:38s 	Training Loss: 0.677547 	Validation Loss: 0.697075
Validation loss decreased. Train loss: 0.677547 Validation Loss: (0.701134 --> 0.697075).  Saving model ...
Epoch: 6 Tiempo:37s 	Training Loss: 0.676715 	Validation Loss: 0.695158
Validation loss decreased. Train loss: 0.676715 Validation Loss: (0.697075 --> 0.695158).  Saving model ...
Epoch: 7 Tiempo:38s 	Training Loss: 0.674884 	Validation Loss: 0.692564
Validation loss decreased. Train loss: 0.674884 Validation Loss: (0.6